# Introduction to data science
Author: Gérard Lichtert

## Introduction
This notebook is to clean data from a csv, it removes unnecesary columns, computes means and saves the processed data to a new csv file found in the output folder.

It will also make a new dataframe containing averages per day per participant and save it to a csv for the OBSE survey

## Usage
Following the instructions in the README.md file is crucial for installation, for execution it suffices to run all twice. The first time will create the necessary directories. You are expected to put the input CSV files in the resources/in directory. After you've moved the CSV files to the resources/in directory you can run it again to process the CSV files. After processing they will end up in the resources/out directory and depending on the data processed in either obse or afvar subdirectories.

## Variables you can change
In the following code cells you can change the variables as you need as these will be the columns that need to be removed from the OBSE survey and the other one respectively.

In [1]:
# This is a list of headers we want to delete (exluding the ones with _TZ, _RT and _TZ) from the OBSE survey
# the headers with _TZ, _RT and _TZ will be removed automatically.
HEADERS_TO_DROP_FINAL_SURVEY: list[str] = [
    "Start Date",
    "End Date",
    "Response Type",
    "IP Address",
    "Progress",
    "Duration (in seconds)",
    "Finished",
    "Recorded Date",
    "Response ID",
    "Recipient Last Name",
    "Recipient First Name",
    "External Data Reference",
    "Location Latitude",
    "Location Longitude",
    "Distribution Channel",
    "User Language",
    'Recipient Email',
]

In [2]:
# This is a list of headers we want to delete (exluding the ones with _TZ, _RT and _TZ) from the afvar survey
# the headers with _TZ, _RT and _TZ will be removed automatically.
HEADERS_TO_DROP_OTHER = [
    "STUDY_ID",
    "STUDY_NAME",
    "STUDY_VERSION",
    "SURVEY_ID",
    "TRIGGER",
    "EXPORT_TZ",
    "START_END",
    "CREATED_TS",
    "SCHEDULED_TS",
    "STARTED_TS",
    "EXPIRED_TS",
    "TOTAL_RT",
    "RAND_PROB",
    "PARTICIPANT_TZ",
]

In [3]:
HEADERS_TO_DROP_FROM_DEMOGRAPHICS: list[str] = [
    "Start Date",
    "End Date",
    "Response Type",
    "IP Address",
    "Progress",
    "Duration (in seconds)",
    "Finished",
    "Recorded Date",
    "Response ID",
    "Recipient Last Name",
    "Recipient First Name",
    "Recipient Email",
    "External Data Reference",
    "Location Latitude",
    "Location Longitude",
    "Distribution Channel",
    "User Language",
    "Beste participant,\r\n\r\nHartelijk dank voor uw deelname aan dit onderzoek.\r\n\r\n\r\n\r\nHet onderzoek\r\n\r\nhet onderzoek bestaat uit twee delen en peilt naar de relatie met uw werk. Deze enquête vormt het eerste deel van het onderzoek. De enquête bevat tien vragen en neemt ongeveer één minuut in beslag. Deze enquête gaat na of u in aanmerking komt voor het tweede deel van het onderzoek betreffende de relatie met uw werk. We vragen uw e-mailadres om u vervolgens een uitnodiging tot de app SEMA3 te sturen. Via deze app zal u het tweede deel van het onderzoek kunnen vervolledigen. \r\n\r\n\r\n\r\nProcedure\r\n\r\nHet onderzoek zelf of het tweede deel betreft een dagboekonderzoek. Hierbij zal u 20 werkdagen lang om 18h een vragenlijst krijgen, waarin u bevraagd zal worden over uw werkdag. Het invullen van deze vragenlijst zal niet langer duren dan 1 minuut. U heeft de tijd tot 24h om deze vragenlijst in te vullen. Participanten die aan 80% of meer meetmomenten deelnamen krijgen een vergoeding voor hun participatie.\r\n\r\n\r\n\r\nWat gebeurt er met mijn gegevens?\r\n\r\nDe verzamelde gegevens worden alleen gebruikt voor wetenschappelijke doeleinden en worden vertrouwelijk behandeld in overeenstemming met de Europese Algemene Verordening Gegevensbescherming (GDPR). Uw deelname is vrijwillig en u kunt op elk moment beslissen om het onderzoek zonder het geven van een reden te beëindigen. Voor meer informatie over uw rechten en de behandeling van de gegevens kunt u contact opnemen met de verantwoordelijke afdeling aan de VUB (dpo@vub.be).\r\n\r\n\r\n\r\nContact\r\n\r\nAls u vragen en/of opmerkingen hebt over dit onderzoek, kunt u ze hieronder meegeven of contact opnemen met Sam de Pape (Sam.De.Pape@vub.be) of Jules Joukes (Jules.Sabine.P.Joukes@vub.be), of met superviserend professor Joeri Hofmans (Joeri.Hofmans@vub.be).",
    "Door deze enquête in te vullen, ga ik akkoord met mijn deelname aan dit onderzoek en met de verwerking van mijn persoonlijke gegevens in overeenstemming met de Europese Algemene Verordening Gegevensbescherming (GDPR) door de onderzoekers van de VUB.",
    "Gaat u volgende maand minstens één week op vakantie? - Selected Choice",
    "Gaat u volgende maand minstens één week op vakantie? - Andere, namelijk: - Text",
]

In [4]:
OBSE_COLUMNS = [
    "PARTICIPANT_ID",
    "UPLOADED_TS",
    "ACTIVITEIT",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "10",
]


## Some libraries and definitions (do not change)


In [5]:
from pathlib import Path
import polars as pl

resources = Path("../resources/in/")
out = Path("../resources/out/")


def write_csv_and_excel(lf, filename):
    lf.sink_csv(f"{out.as_posix()}/{filename}.csv")
    lf.collect().write_excel(f"{out.as_posix()}/{filename}.xlsx")


## reading and cleaning the data

In [6]:
final_survey_lf: pl.LazyFrame = pl.scan_csv(f"{resources.as_posix()}/final_survey.csv")
sema_lf: pl.LazyFrame = pl.scan_csv(
    resources.as_posix() + "/" + "data_uit_SEMA3_OBSE_en_laatste_survey.csv"
)
demographics_lf: pl.LazyFrame = pl.scan_csv(
    resources.as_posix() + "/" + "demografische_gegevens_eerste_survey.csv"
)
keys: pl.LazyFrame = pl.scan_csv(
    f"{resources.as_posix()}/identificatie_key.csv", separator=";"
)


def remove_headers(
    lf: pl.LazyFrame, headers: list[str], del_timed_headers=True
) -> pl.LazyFrame:
    columns: list[str] = lf.columns
    keep = [col for col in columns if col not in headers]
    if del_timed_headers:
        keep = [
            col
            for col in keep
            if not col.endswith("_TZ")
            and not col.endswith("_RT")
            and not col.endswith("_TZ")
        ]
    return lf.select(keep)


demographics_lf = remove_headers(
    demographics_lf, HEADERS_TO_DROP_FROM_DEMOGRAPHICS, False
)
final_survey_lf = remove_headers(final_survey_lf, HEADERS_TO_DROP_FINAL_SURVEY, False)
sema_en_obse_lf = remove_headers(sema_lf, HEADERS_TO_DROP_OTHER)
obse_lf: pl.LazyFrame = sema_en_obse_lf.filter(pl.col("SURVEY_NAME") == "OBSE")
werktevredenheid_lf: pl.LazyFrame = sema_en_obse_lf.filter(
    pl.col("SURVEY_NAME") == "werktevredenheid/ SWLS/ PRESTATIE"
)

In [7]:
obse_lf: pl.LazyFrame = (
    obse_lf.select(OBSE_COLUMNS)
    .filter(pl.col("ACTIVITEIT") == "1")
    .cast({str(integer): pl.UInt8 for integer in range(1, 11)})
)
valid_obse_participants: list[str] = (
    obse_lf.group_by("PARTICIPANT_ID")
    .len()
    .filter(pl.col("len") >= 5)
    .select("PARTICIPANT_ID")
    .unique()
    .collect()
    .to_dict(as_series=False)["PARTICIPANT_ID"]
)
obse_lf = (
    obse_lf.filter(pl.col("PARTICIPANT_ID").is_in(valid_obse_participants))
    .with_columns(
        (
            (
                pl.col("1")
                + pl.col("2")
                + pl.col("3")
                + pl.col("4")
                + pl.col("5")
                + pl.col("6")
                + pl.col("7")
                + pl.col("8")
                + pl.col("9")
                + pl.col("10")
            )
            / 10
        ).alias("MEAN")
    )
    .sort("UPLOADED_TS")
)
write_csv_and_excel(obse_lf, "obse_with_daily_mean")
obse_participant_mean = obse_lf.group_by("PARTICIPANT_ID").agg(pl.col("MEAN").mean())
write_csv_and_excel(obse_participant_mean, "obse_participant_mean")

In [14]:
final_survey: pl.LazyFrame = (
    final_survey_lf.join(
        keys, left_on="Wat is uw e-mailadres dat u opgaf voor SEMA3?", right_on="Email"
    )
    .rename(
        {
            "Wat is uw e-mailadres dat u opgaf voor SEMA3?": "EMAIL",
            "Id": "PARTICIPANT_ID",
        }
    )
    .drop(["EMAIL"])
)
# final_survey.collect().head(50)
werktevredenheid_lf.drop([str(integer) for integer in range(1, 11)]).filter(~pl.all_horizontal(pl.all().is_null())).columns


['PARTICIPANT_ID',
 'SURVEY_NAME',
 'COMPLETED_TS',
 'UPLOADED_TS',
 'INLEIDING_WERKTEVREDENHEID',
 'WERKOMSTANDIGHEDEN_1',
 'WERKOMSTANDIGHEDEN_2',
 'WERKOMSTANDIGHEDEN_3',
 'WERKOMSTANDIGHEDEN_4',
 'WERKOMSTANDIGHEDEN_5',
 'WERKMETHODE_1',
 'WERKMETHODE_2',
 'WERKMETHODE_3',
 'WERKMETHODE_4',
 'WERKMETHODE_5',
 "COLLEGA'S_1",
 "COLLEGA'S_2",
 "COLLEGA'S_3",
 "COLLEGA'S_4",
 "COLLEGA'S_5",
 'ERKENNING_1',
 'ERKENNING_2',
 'ERKENNING_3',
 'ERKENNING_4',
 'ERKENNING_5',
 'BAAS_1',
 'BAAS_2',
 'BAAS_3',
 'BAAS_4',
 'BAAS_5',
 'VERANTWOORDELIJKHEID_1',
 'VERANTWOORDELIJKHEID_2',
 'VERANTWOORDELIJKHEID_3',
 'VERANTWOORDELIJKHEID_4',
 'VERANTWOORDELIJKHEID_5',
 'SALARIS_1',
 'SALARIS_2',
 'SALARIS_3',
 'SALARIS_4',
 'SALARIS_5',
 'CAPACITEITEN_1',
 'CAPACITEITEN_2',
 'CAPACITEITEN_3',
 'CAPACITEITEN_4',
 'CAPACITEITEN_5',
 'ARBEIDSVERHOUDING_1',
 'ARBEIDSVERHOUDING_2',
 'ARBEIDSVERHOUDING_3',
 'ARBEIDSVERHOUDING_4',
 'ARBEIDSVERHOUDING_5',
 'PROMOTIE_1',
 'PROMOTIE_2',
 'PROMOTIE_3',
 'PROM

PARTICIPANT_ID,SURVEY_NAME,COMPLETED_TS,UPLOADED_TS,INLEIDING_WERKTEVREDENHEID,WERKOMSTANDIGHEDEN_1,WERKOMSTANDIGHEDEN_2,WERKOMSTANDIGHEDEN_3,WERKOMSTANDIGHEDEN_4,WERKOMSTANDIGHEDEN_5,WERKMETHODE_1,WERKMETHODE_2,WERKMETHODE_3,WERKMETHODE_4,WERKMETHODE_5,COLLEGA'S_1,COLLEGA'S_2,COLLEGA'S_3,COLLEGA'S_4,COLLEGA'S_5,ERKENNING_1,ERKENNING_2,ERKENNING_3,ERKENNING_4,ERKENNING_5,BAAS_1,BAAS_2,BAAS_3,BAAS_4,BAAS_5,VERANTWOORDELIJKHEID_1,VERANTWOORDELIJKHEID_2,VERANTWOORDELIJKHEID_3,VERANTWOORDELIJKHEID_4,VERANTWOORDELIJKHEID_5,SALARIS_1,SALARIS_2,…,2,3,4,5,6,7,8,9,10,BEDANKT,TEGENVALLEND_RESULTAAT),TEGENVALLEND_RESULTAAT)_CLONE_e5KL51Ni-,CONTROLLEVRAAG,TAAKPRESTATIE,INLEIDING_CLONE_fR1ZU640OP,1_CLONE_9wDf9fDsLv,2_CLONE_aC3fFpRTBJ,3_CLONE_R18jp8h9BU,4_CLONE_4li3npRZEb,5_CLONE_igbEobmcN7,6_CLONE_0oBqe2bw0k,7_CLONE_TIIvEor2lb,8_CLONE_0s_HcRRV8G,9_CLONE_wTSgFrNeVm,10_CLONE_rVAhNpsMA2,"""IDEAAL""","""IDEAAL""""_R","""OMSTANDIGHEDEN""","""OMSTANDIGHEDEN""""_R","""TEVREDEN""","""TEVREDEN""""_R","""BELANGRIJKE_DINGEN""","""BELANGRIJKE_DINGEN""""_R","""NIETS_VERANDEREN""","""NIETS_VERANDEREN""""_R",INTRO,SLOT
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""s257136850""","""werktevredenheid/ SWLS/ PRESTA…","""08-May-2024 10:03""","""08-May-2024 10:03""",null,"""0""","""1""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""1""","""1""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""",…,"""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""5""","""5622""","""5""","""2460""","""5""","""1579""","""5""","""3986""","""4""","""2022""","""<no-response>""","""<no-response>"""
"""s272728106""","""werktevredenheid/ SWLS/ PRESTA…","""08-May-2024 17:04""","""08-May-2024 17:04""",null,"""0""","""0""","""1""","""0""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""","""1""","""0""","""0""","""1""","""0""","""0""","""0""","""0""","""0""","""0""",…,"""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""6""","""5732""","""6""","""3011""","""7""","""2260""","""7""","""3812""","""7""","""3161""","""<no-response>""","""<no-response>"""
"""s273278439""","""werktevredenheid/ SWLS/ PRESTA…",null,null,"""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-response>""","""<no-respo